In [ ]:
%pip install pandas
%pip install openai
%pip install python-dotenv
%pip install scikit-learn

In [28]:
from openai import OpenAI
import os
import json
from sklearn.metrics.pairwise import cosine_similarity
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
def get_chatbot_response(client, model_name, messages, temperature=0.0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000
    ).choices[0].message.content

    return response 

In [9]:
client = OpenAI(
    api_key=os.getenv('RUNPOD_TOKEN'),
    base_url=os.getenv('RUNPOD_CHATBOT_URL'),
)
model_name = os.getenv("MODEL_NAME")

# Get LLM response

In [ ]:
messages = [{"role": "user", "content": "What is the capital of Germany?"}]
response = get_chatbot_response(client,model_name,messages)

In [ ]:
response

# Prompt Engineering

## Structured output

In [ ]:
system_prompt = """
You are a helpful assistant that answers questions about capitals of countries.

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
[
{
    "country": the country that you will get the capital of,
    "capital": the capital of the country stated
}
]
"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": "What is the capital of Germany?"})
response = get_chatbot_response(client,model_name,messages)
print(response)


In [ ]:
type(response)

In [ ]:
json_response = json.loads(response)
json_response

In [ ]:
type(json_response[0]),json_response[0]['capital']

## Input Structuring

In [ ]:
user_input = """
Get me the capitals of the following countries:
```
1. Italy
2. Germany
3. France
```
"""

messages = [{"role": "system", "content": system_prompt}]
messages.append({"role": "user", "content": user_input})
response = get_chatbot_response(client,model_name,messages)
print(response)

In [ ]:
json_response = json.loads(response)
json_response

## Give the model time to think (Chain of thought)

In [ ]:
user_prompt = """
Calculate the result of this equation: 1+3

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{"role": "system", "content": user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

In [ ]:
259/2*8654+91072*33-12971

In [ ]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{
    result: The final number resulted from calculating the equation above
}
"""

messages = [{"role": "system", "content": user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

In [15]:
4113098.0 - 1343519

2769579.0

In [ ]:
user_prompt = """
Calculate the result of this equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{
    steps: This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to the final result. Feel free to write in free text.
    result: The final number resulted from calculating the equation above
}
"""

messages = [{"role": "system", "content": user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

In [14]:
4113098.0 - 4005806.5

107291.5

# RAG - Retrival Augmented Generation

In [ ]:
user_prompt = """
What's new in iphone 16?
"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

In [20]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [ ]:
user_prompt = """
{iphone_16}

What's new in iphone 16?
"""

messages = [{"role": "user", "content": user_prompt}]
response = get_chatbot_response(client,model_name,messages)
print(response)

### Automatically Extract context data from database

In [23]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [24]:
data = [iphone_16, samsung_s23]


In [25]:
user_prompt = """What's new in iphone 16?"""

In [26]:
embedding_client = OpenAI(
    api_key=os.getenv('RUNPOD_TOKEN'), 
    base_url=os.getenv('RUNPOD_EMBEDDING_URL')
)

In [ ]:
def get_embedding(embedding_client, model_name, text_input):
    output = embedding_client.embeddings.create(input = text_input, model=model_name)
    
    embedings = []
    for embedding_object in output.data:
        embedings.append(embedding_object.embedding)

    return embedings

In [ ]:
user_prompt_embedding = get_embedding(embedding_client, model_name, user_prompt)[0]

In [ ]:
data_embeddings = [get_embedding(embedding_client, model_name, text)[0] for text in data]

In [ ]:
data_similarity = cosine_similarity([user_prompt_embedding], data_embeddings)

In [ ]:
data_similarity

In [ ]:
closest_entry_index = data_similarity.argmax()
closest_entry_index

In [ ]:
data[closest_entry_index]

In [ ]:
user_prompt_with_data = f"""
{data[closest_entry_index]}

{user_prompt}
"""

In [ ]:
print(user_prompt_with_data)

In [ ]:
messages = [{"role": "user", "content": user_prompt_with_data}]
response = get_chatbot_response(client,model_name,messages)
print(response)